In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
debug_file = "../output/res_16.v2.5.debug.final.csv"

In [3]:
debug_df = pd.read_csv(debug_file)

In [4]:
debug_df.columns

Index(['id', 'after', 'before', 'class_pred', 'max_prob'], dtype='object')

In [5]:
debug_df["class_pred"].value_counts()

PLAIN         688297
PUNCT         193493
DATE           27004
LETTERS        17267
CARDINAL       13159
VERBATIM        9918
MEASURE         2518
ORDINAL         1114
ELECTRONIC       815
DECIMAL          797
MONEY            532
DIGIT            461
TELEPHONE        352
TIME             151
FRACTION         106
ADDRESS           62
Name: class_pred, dtype: int64

In [6]:
debug_df.head()

,id,after,before,class_pred,max_prob
0,0_0,Last,Last,PLAIN,0.999873
1,0_1,modified,modified,PLAIN,0.999985
2,0_2,the thirty first of march twenty sixteen,2016-03-31,DATE,0.999987
3,0_3,.,.,PUNCT,1.000000
4,1_0,There's,There's,PLAIN,0.999987


检查是否有数字

In [7]:
def has_digit(token):
    token = str(token)
    for c in token:
        if c.isdigit():
            return True
    return False

In [8]:
debug_df.loc[debug_df["after"].apply(has_digit)]

,id,after,before,class_pred,max_prob
8746,702_11,c b ₁,CB₁,LETTERS,0.903700
8751,702_16,c b ₂,CB₂,LETTERS,0.979861
13717,1084_10,¹,¹,VERBATIM,0.992725
19832,1547_3,₂,₂,VERBATIM,0.996975
26827,2086_1,ZnI₂,ZnI₂,PLAIN,0.892235
32076,2489_8,18000000square,"18,000,000 square",CARDINAL,0.891726
45313,3502_13,CaCO3,CaCO3,PLAIN,0.529404
61114,4713_6,NH₄,NH₄,PLAIN,0.979966
61116,4713_8,₂,₂,VERBATIM,0.999373
61118,4713_10,₂,₂,VERBATIM,0.897784


检查是否有符号

In [9]:
def has_symbol(token):
    token = str(token)
    for c in token:
        if c.isdigit() or c.isalpha() or c == " ":
            continue
        else:
            return True
    return False

In [10]:
debug_df.loc[np.logical_and(np.logical_and(debug_df["after"].apply(has_symbol), debug_df["class_pred"] != "PUNCT"), debug_df["class_pred"] != "PLAIN")][50:100]

,id,after,before,class_pred,max_prob
26384,2051_5,-,-,VERBATIM,0.999981
26697,2076_2,-,-,VERBATIM,0.999988
26808,2084_10,n u's,Nus,LETTERS,0.618101
26838,2086_12,-,-,VERBATIM,0.999093
27626,2149_11,-,-,VERBATIM,0.999983
28045,2184_3,-,-,VERBATIM,0.999970
28249,2198_14,-,-,VERBATIM,0.999979
28589,2223_6,-,-,VERBATIM,0.999926
30310,2350_6,-,-,VERBATIM,0.999917
30314,2350_10,-,-,VERBATIM,0.999931


检查信心不高的分类

In [9]:
debug_df["max_prob"].value_counts(bins=10)

(0.934, 1.0]      946833
(0.868, 0.934]      3246
(0.801, 0.868]      1666
(0.735, 0.801]      1201
(0.669, 0.735]       908
(0.603, 0.669]       845
(0.536, 0.603]       797
(0.47, 0.536]        477
(0.404, 0.47]         44
(0.336, 0.404]        29
Name: max_prob, dtype: int64

In [10]:
debug_df.loc[153008]

id                       11598_2
after         the sixteenth june
before             the 16th June
class_pred                  DATE
max_prob                0.999785
Name: 153008, dtype: object

In [11]:
debug_df.loc[np.logical_and(debug_df["max_prob"] < 0.404, debug_df["max_prob"] > 0.336)]

,id,after,before,class_pred,max_prob
4852,398_15,forty five years,45 years,MEASURE,0.358373
21990,1702_12,fifteen years,15 years,MEASURE,0.357256
51711,3994_3,twenty kilometres,20 kilometres,MEASURE,0.350195
103283,7893_1,seventy four years,74 years,MEASURE,0.398179
112576,8591_5,one hundred five point five miles per hour,105.5 miles per hour,MEASURE,0.394667
170206,12861_4,thirty years,30 years,MEASURE,0.364446
210893,15884_11,twelve years,12 years,MEASURE,0.360964
221582,16670_1,thirty minutes,30 minutes,MEASURE,0.373131
326168,24276_13,the first of december,01 Dec,DATE,0.402090
353055,26233_6,eighteen years,18 years,MEASURE,0.345881


In [12]:
debug_df.loc[np.logical_and(debug_df["max_prob"] < 0.47, debug_df["max_prob"] > 0.404)]

,id,after,before,class_pred,max_prob
43271,3345_8,m p,mp,LETTERS,0.438912
66274,5098_4,two o o o,2000,DIGIT,0.443002
76726,5891_13,thirty six weeks,36 weeks,MEASURE,0.422542
86249,6604_7,ten kilometers,10 kilometers,MEASURE,0.410027
110291,8421_8,twenty two years,22 years,MEASURE,0.457991
115592,8809_6,seventy five years,75 years,MEASURE,0.406574
146855,11148_7,three thirty seven thousand five hundred sixths,3/37506,CARDINAL,0.408591
149500,11339_2,eleven years,11 years,MEASURE,0.436643
179256,13523_7,one hundred ten kilometres,110 kilometres,MEASURE,0.462128
181616,13691_9,x l s,xls,VERBATIM,0.435031


In [16]:
debug_df.loc[np.logical_and(debug_df["max_prob"] < 0.536, debug_df["max_prob"] > 0.47)]

,id,after,before,class_pred,max_prob
263,25_11,DID,DID,PLAIN,0.506123
965,84_1,a e t,AET,LETTERS,0.528986
5746,473_3,i g,Ig,LETTERS,0.535981
6456,528_9,p,p,PLAIN,0.532378
7350,598_4,GATA,GATA,PLAIN,0.500221
7822,630_8,OPALS,OPALS,LETTERS,0.529580
9375,755_21,IEEE,IEEE,PLAIN,0.529684
10006,801_12,r o c o r,ROCOR,LETTERS,0.518825
10784,859_1,Encyclopédisque,Encyclopédisque,PLAIN,0.519043
13186,1043_11,semi-finalist,semi-finalist,PLAIN,0.501891


In [6]:
mea_df = debug_df.loc[debug_df["class_pred"] == "MEASURE"]

In [6]:
def is_one(token):
    token = str(token)
    if " per " in token:
        for one in ["per square"]:
            if one in token:
                return False
        return True
#     if token.startswith("one ") and token.startswith("s"):
#         return True
    return False

In [7]:
debug_df.loc[np.logical_and(debug_df["class_pred"] == "MEASURE", debug_df["before"].apply(lambda s: str(s).endswith("s")))]

,id,after,before,class_pred
958,82_13,six years,6 years,MEASURE
1163,101_7,one point five centimetres,1.5 centimetres,MEASURE
1573,137_7,twenty eight years,28 years,MEASURE
2747,230_3,seven thousand acres,"7,000 acres",MEASURE
2749,230_5,two thousand eight hundred hectares,"2,800 hectares",MEASURE
3034,254_4,ninety thousand tons,"90,000 tons",MEASURE
4852,398_15,forty five years,45 years,MEASURE
8301,665_2,six years,6 years,MEASURE
9170,737_15,forty five minutes,45 minutes,MEASURE
9172,737_17,six hours,6 hours,MEASURE


In [9]:
debug_df.loc[debug_df["after"].apply(is_one)]

,id,after,before,class_pred
24501,1904_6,seventeen kilo joules per grams,17 kJ/g,MEASURE
24503,1904_8,four kilo calories per grams,4 kcal/g,MEASURE
83431,6402_19,two hundred sixty kilo joules per cubic meters,260 kJ/m³,MEASURE
86428,6619_11,two hundred fifty kilometers per hour,250 km/h,MEASURE
86430,6619_13,one hundred sixty miles per hour,160 mph,MEASURE
92782,7111_19,fifteen n grams per deciliters,15 ng/dL,MEASURE
94996,7274_4,sixty miles per hour,60 mph,MEASURE
94998,7274_6,ninety seven kilometers per hour,97 km/h,MEASURE
106005,8103_5,twenty seven point eight l meters grams per mi...,27.8 mg/ml,MEASURE
112576,8591_5,one hundred five point five per our miles hour,105.5 miles per hour,MEASURE


In [4]:
debug_df["class_pred"].value_counts()

PLAIN         688249
PUNCT         193491
DATE           27001
LETTERS        17317
CARDINAL       13162
VERBATIM        9920
MEASURE         2518
ORDINAL         1113
ELECTRONIC       815
DECIMAL          793
MONEY            532
DIGIT            462
TELEPHONE        353
TIME             152
FRACTION         106
ADDRESS           62
Name: class_pred, dtype: int64

In [20]:
debug_df.loc[debug_df["class_pred"] == "FRACTION"][50:]

,id,after,before,class_pred
551773,40712_19,eleven fifteenths,11/15,FRACTION
570080,42042_6,nine hundred eleven halves,911/2,FRACTION
572993,42254_6,one thousand seven hundred twenty one twenty s...,1721/22,FRACTION
575936,42460_8,one thousand eight hundred seventy ninety firsts,1870/91,FRACTION
575986,42463_15,minus one quarter,-1/4,FRACTION
576067,42470_4,minus one half,-1/2,FRACTION
578209,42622_18,fifty nine sixty firsts,59/61,FRACTION
583873,43037_12,fifth,5º,FRACTION
585029,43126_9,two hundred seventy two thousand fifteenths,270/2015,FRACTION
588066,43346_3,two fifths,2/5,FRACTION


In [6]:
debug_df.loc[debug_df["class_pred"] == "DATE"]

,id,after,before,class_pred
2,0_2,the thirty first of march twenty sixteen,2016-03-31,DATE
37,3_11,april twenty seventeen,April 2017,DATE
43,3_17,may twenty seventeen,May 2017,DATE
45,4_0,the twenty first of february twenty seventeen,21 February 2017,DATE
151,14_1,the fourth of march twenty seventeen,4 March 2017,DATE
165,16_1,march twenty eighth twenty seventeen,"March 28, 2017",DATE
205,20_1,april eighteenth twenty seventeen,"April 18, 2017",DATE
250,24_0,march ninth twenty seventeen,"March 9, 2017",DATE
253,25_1,twenty fifteen,2015,DATE
282,26_15,nineteen eighty two,1982,DATE


In [7]:
debug_df.head()

,id,after,before,class_pred
0,0_0,Last,Last,PLAIN
1,0_1,modified,modified,PLAIN
2,0_2,the thirty first of march twenty sixteen,2016-03-31,DATE
3,0_3,.,.,PUNCT
4,1_0,There's,There's,PLAIN


In [12]:
SUB = str.maketrans("₀₁₂₃₄₅₆₇₈₉", "0123456789")
SUP = str.maketrans("⁰¹²³⁴⁵⁶⁷⁸⁹", "0123456789")
OTH = str.maketrans("፬", "4")
strange_list = list("₀₁₂₃₄₅₆₇₈₉⁰¹²³⁴⁵⁶⁷⁸⁹፬")
def is_strange(token):
    token = str(token)
    for c in strange_list:
        if c in token:
            return True
    return False

In [18]:
debug_df_2.loc[debug_df_2["before"] == "CO2"]

,id,after,before,class_pred


In [14]:
debug_df_2.loc[np.logical_and(debug_df_2["after"].apply(is_strange), debug_df_2["class_pred"] != "VERBATIM")][:50]

,id,after,before,class_pred
3036,254_6,c o ₂,CO₂,LETTERS
8746,702_11,c b ₁,CB₁,LETTERS
8751,702_16,c b ₂,CB₂,LETTERS
26827,2086_1,ZnI₂,ZnI₂,PLAIN
61114,4713_6,n h ₄,NH₄,LETTERS
61129,4713_21,SiO₂,SiO₂,PLAIN
68839,5294_2,³,³,PUNCT
122376,9307_5,OH)₂,OH)₂,PLAIN
171822,12983_13,c h ₃,CH₃,LETTERS
182372,13747_18,CaCO₃,CaCO₃,PLAIN


In [14]:
def has_measure_string(token):
    token = str(token)
    mea_strs = ['years', 'year', 'months', 'month', 'weeks', 'week', 'days', 'day',
                'hours', 'minutes', 'min', 'seconds', 'nautical', 'acres', 'hectares',
                'tonnes', 'tons', 'kilobytes', 'nanobarns', 'square', 'miles', 'kilometres',
                'kilometre', 'kilometers', 'kilometer', 'meters', 'meter', 'metres', 'metre',
                'feet', 'yards', 'barrels', 'calories', 'watts', 'knots', 'inches', 'pounds',
                'degrees', 'bar', 'Cellos']
    for one in mea_strs:
        if one in token:
            return True
    return False

mea_str_df = debug_df_2.loc[debug_df_2["before"].apply(has_measure_string)]

In [18]:
mea_str_df.loc[np.logical_and(mea_str_df["class_pred"] != "PLAIN", mea_str_df["class_pred"] != "MEASURE")]["class_pred"].value_counts()

DATE          165
MONEY          25
ELECTRONIC     16
VERBATIM       10
CARDINAL        1
TELEPHONE       1
Name: class_pred, dtype: int64

In [22]:
mea_str_df.loc[mea_str_df["class_pred"] == "MONEY"]

,id,after,before,class_pred
1163,101_7,1.5 centimetres,1.5 centimetres,MONEY
74417,5729_12,3.18 square,3.18 square,MONEY
86249,6604_7,ten kilomete rupees,10 kilometers,MONEY
140961,10708_9,"33,644 tons","33,644 tons",MONEY
155361,11773_10,4.4 million barrels,4.4 million barrels,MONEY
162680,12309_9,"80,000 square","80,000 square",MONEY
164021,12405_16,fifteen million yea rupees,15 million years,MONEY
170469,12879_16,ninety three million yea rupees,93 million years,MONEY
170472,12879_19,sixty eight thousand yea rupees,"68,000 years",MONEY
234740,17609_5,ninety six million yea rupees,96 million years,MONEY


In [8]:
debug_df_2.loc[debug_df_2["after"].apply(lambda s: "odd!~!" in str(s))][:50]

,id,after,before,class_pred
166872,12610_18,odd!~! four hundredth zero zero,"E£ 400,000",DATE
457101,33828_8,the fourth of odd!~! twenty seventeen,4 16 2017,DATE
537491,39677_9,the eight hundred and eightieth of odd!~!,880 CE.,DATE
769590,56482_6,the tenth of odd!~!,10 4-6-2,DATE
